# ResNet

> Neural net model

In [ ]:
#| default_exp models.resnet

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
import torch.nn as nn

import torch
from torchinfo import summary
from torchvision.transforms import transforms


from omegaconf import OmegaConf
from hydra.utils import instantiate

from matplotlib import pyplot as plt
import math

from nimrod.models.conv import ConvLayer
from nimrod.models.core import Classifier
from nimrod.utils import get_device, set_seed
from nimrod.image.datasets import ImageDataModule

from typing import List, Optional, Callable, Any
import logging
from functools import partial


In [ ]:
#| export
logger = logging.getLogger(__name__)
set_seed()

Seed set to 42


## Res Block

In [ ]:
#| export 
class ResBlock(nn.Module):
    def __init__(
            self,
            n_channels: int=3 # Number of input & output channels
        ):

        super().__init__()

        layers = []
        conv = partial(ConvLayer, n_channels, n_channels, stride=1, normalization=nn.BatchNorm2d)
        layers += [conv(activation=nn.ReLU), conv(activation=None)]
        self.layers = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x + self.layers(x)

### Usage

In [ ]:
model = ResBlock(3)
x = torch.randn(1, 3, 32, 32)
y = model(x)
print(y.shape)
summary(model=model, input_size=(1, 3, 32, 32), depth=3)

[15:34:36] WARNING - setting conv bias to False as Batchnorm is used
[15:34:36] WARNING - setting conv bias to False as Batchnorm is used


torch.Size([1, 3, 32, 32])


Layer (type:depth-idx)                   Output Shape              Param #
ResBlock                                 [1, 3, 32, 32]            --
├─Sequential: 1-1                        [1, 3, 32, 32]            --
│    └─ConvLayer: 2-1                    [1, 3, 32, 32]            --
│    │    └─Sequential: 3-1              [1, 3, 32, 32]            87
│    └─ConvLayer: 2-2                    [1, 3, 32, 32]            --
│    │    └─Sequential: 3-2              [1, 3, 32, 32]            87
Total params: 174
Trainable params: 174
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.17
Input size (MB): 0.01
Forward/backward pass size (MB): 0.10
Params size (MB): 0.00
Estimated Total Size (MB): 0.11

## ResNet

In [ ]:
#| export
class ResNet(nn.Module):
    def __init__(
            self,
            n_features: List[int]=[1, 8, 16, 32, 16], # Number of input & output channels
            num_classes: int=10, # Number of classes
        ):

        super().__init__()
        logger.info("ResNet: init")
        layers = []
        conv = partial(ConvLayer, stride=2, normalization=nn.BatchNorm2d, activation=nn.ReLU)
        # convnet with resblock between
        for i in range(len(n_features)-1):
            layers += [conv(n_features[i], n_features[i+1])]
            layers += [ResBlock(n_features[i+1])]

        # last layer back to n_classes and flatten
        layers.append(conv(n_features[-1], num_classes))
        layers.append(nn.Flatten())
        self.layers = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layers(x)

In [ ]:
x = torch.randn(64, 1, 28, 28)
model = ResNet(n_features=[1, 8, 16, 32, 16], num_classes=10)
y = model(x)
print(y.shape)

[15:35:47] INFO - ResNet: init
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used
[15:35:47] WARNING - setting conv bias to False as Batchnorm is used


torch.Size([64, 10])


## ResNetX


In [ ]:
#| export

class ResNetX(Classifier):
    def __init__(
        self,
        nnet:ResNet,
        num_classes:int,
        optimizer:Callable[...,torch.optim.Optimizer], # optimizer,
        scheduler: Optional[Callable[...,Any]]=None, # scheduler
        ):
        
        logger.info("ResNetX: init")
        super().__init__(
            nnet=nnet,
            num_classes=num_classes,
            optimizer=optimizer,
            scheduler=scheduler
            )

    def _step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        preds = y_hat.argmax(dim=1)
        return loss, preds, y
    
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat = self.forward(x)
        return y_hat.argmax(dim=1)

### Usage

In [ ]:
cfg = OmegaConf.load('../config/model/image/resnetx.yaml')
B, C, H, W = 64, 1, 28, 28
x = torch.randn(B, C, H, W)
nnet = instantiate(cfg.nnet, num_classes=10)
y = nnet(x)
print(y.shape)


[15:40:37] INFO - ResNet: init
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used


torch.Size([64, 10])


In [ ]:
summary(nnet, input_size=(B, C, H, W), depth=5)

Layer (type:depth-idx)                             Output Shape              Param #
ResNet                                             [64, 10]                  --
├─Sequential: 1-1                                  [64, 10]                  --
│    └─ConvLayer: 2-1                              [64, 8, 14, 14]           --
│    │    └─Sequential: 3-1                        [64, 8, 14, 14]           --
│    │    │    └─Conv2d: 4-1                       [64, 8, 14, 14]           72
│    │    │    └─BatchNorm2d: 4-2                  [64, 8, 14, 14]           16
│    │    │    └─ReLU: 4-3                         [64, 8, 14, 14]           --
│    └─ResBlock: 2-2                               [64, 8, 14, 14]           --
│    │    └─Sequential: 3-2                        [64, 8, 14, 14]           --
│    │    │    └─ConvLayer: 4-4                    [64, 8, 14, 14]           --
│    │    │    │    └─Sequential: 5-1              [64, 8, 14, 14]           592
│    │    │    └─ConvLayer: 4-5   

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()